# YvesBlue Backend Challenge
## Carbon Analytic Calculation
### Goal: Utilizing this Jupyter Notebook (and additional libraries/files/tools as needed) implement a function that can be utilized to calculate the (strawman) Adjusted CO2 Total emissions for a company. 
### Notes: The calculation is laid out in this [image](calculation.png?raw=true). The data is stored inside of the [included data file](data.json?raw=true). Your code structure can take whatever you believe to be best (individual function, class, module, etc.). Please implement the code as well as write tests for the calculation, showcasing how you would write tests for such requirements. Provide instructions with your submission for how to call your implemented function as well as how to run the tests.
## Author: @github_username (Full Name)
## Date: YYYY-MM-DD

In [1]:
## Import Libraries

import decimal
import json
import os

import numpy


In [2]:
## Specify Calculation Function

class CarbonAnalytic:
    # def __init__(self):
    def __init__(self, mu_purch=0.5, mu_purch_max=0.8, phi_prod=0.05, print_errors=False):
        self.__keys = {
            "CO2_tot": "Total CO2 Equivalents Emissions",
            "CC": "Carbon Credit Value",
            "RE_purch": "Renewable Energy Purchased",
            "RE_prod": "Renewable Energy Produced",
            "E_tot": "Total Energy Use",
        }

        self.__mu_purch = mu_purch
        self.__mu_purch_max = mu_purch_max
        self.__phi_prod = phi_prod
        self.__print_errors = print_errors

    def calculation_from_file(self, file_path='data.json'):
        ''' To be implemented what should happen if:
            - file is missing
            - cannot open file
            - cannot read from file
            - invalid file format (invalid json)
        '''
        return self.calculation_from_list(json.loads(open(file_path, 'r').read()))
        
    def calculation_from_list(self, data_list):
        for data in data_list:
            data["Adjusted CO2 Total Emissions"] = float(self.calculation_from_data(data))
        return data_list

    def calculation_from_data(self, data):
        try:
            self.is_valid(data)
        except Exception as e:
            if self.__print_errors:
                print(str(e)) # error message
            return None
        
        return (
            decimal.Decimal(data[self.__keys['CO2_tot']]) - decimal.Decimal(data[self.__keys['CC']])
        ) * (
            decimal.Decimal(1) - decimal.Decimal(min(
                decimal.Decimal(self.__mu_purch) * (
                    decimal.Decimal(data[self.__keys['RE_purch']])
                    /
                    decimal.Decimal(data[self.__keys['E_tot']])
                ),
                decimal.Decimal(self.__mu_purch_max)
            ))
        ) - decimal.Decimal(self.__phi_prod) * decimal.Decimal(data[self.__keys['RE_prod']])

    def is_valid(self, data):
        errors = []
        for _k, key in self.__keys.items():
            if key not in data.keys():
                errors.append('Key "{}" is missing from data: {}.'.format(key, data))
                continue
            if type(data[key]) not in (int, float):
                errors.append('Key "{}" has unexpected format: {}.'.format(key, data[key]))
                # continue
        if self.__keys['CO2_tot'] in data.keys() and data[self.__keys['CO2_tot']] == 0:
            errors.append('Key "{}" found with invalid value of {} from data: {}'.format(
                self.__keys['CO2_tot'], data[self.__keys['CO2_tot']], data
            ))
        if len(errors) > 0:
            raise Exception('\n'.join(errors))
        return True


In [3]:
## Demonstrate Solution with Calculation Function

print(json.dumps(CarbonAnalytic(print_errors=True).calculation_from_file(), indent=4)) # pretty print

[
    {
        "ISIN": "US0000000000",
        "Total Energy Use": 7000000,
        "Total CO2 Equivalents Emissions": 94972.49198,
        "Renewable Energy Purchased": 10576.00479,
        "Renewable Energy Produced": 96652.16115,
        "Carbon Credit Value": 8171.323352,
        "Adjusted CO2 Total Emissions": 81902.98845798662
    },
    {
        "ISIN": "US0000000001",
        "Total Energy Use": 80000000,
        "Total CO2 Equivalents Emissions": 306900.6192,
        "Renewable Energy Purchased": 31961.79405,
        "Renewable Energy Produced": 38372.1921,
        "Carbon Credit Value": 57387.1,
        "Adjusted CO2 Total Emissions": 247545.0664717915
    },
    {
        "ISIN": "US0000000002",
        "Total Energy Use": 153000000,
        "Total CO2 Equivalents Emissions": 171320.1651,
        "Renewable Energy Purchased": 62884.91047,
        "Renewable Energy Produced": 10689.83474,
        "Carbon Credit Value": 84303.7259,
        "Adjusted CO2 Total Emissions": 864

In [4]:
## Test Calculation Function

class CarbonAnalyticTest:
    def test_div_by_zero(self):
        assert None == CarbonAnalytic().calculation_from_data(
            {
                "ISIN": "US0000000000",
                "Total Energy Use": 7000000,
                "Total CO2 Equivalents Emissions": 0,
                "Renewable Energy Purchased": 10576.00479,
                "Renewable Energy Produced": 96652.16115,
                "Carbon Credit Value": 8171.323352,
                "Adjusted CO2 Total Emissions": 81902.98845798662
            },
        )
        print('OK - test_div_by_zero')

    def test_with_known_value(self):
        err = decimal.Decimal(10**(-5))
        res = decimal.Decimal(81902.98845798662)
        tst = decimal.Decimal(
            CarbonAnalytic().calculation_from_data(
                {
                    "ISIN": "US0000000000",
                    "Total Energy Use": 7000000,
                    "Total CO2 Equivalents Emissions": 94972.49198,
                    "Renewable Energy Purchased": 10576.00479,
                    "Renewable Energy Produced": 96652.16115,
                    "Carbon Credit Value": 8171.323352,
                },
            )
        )
        assert err > abs(res - tst)
        print('OK - test_with_known_value')

    def test_all(self):
        self.test_div_by_zero()
        self.test_with_known_value()
        # test_from_file()
        # test_with_lists()
        # test_is_valid()
        # test_corner_cases() # if applicable
        # test_precision() # if applicable

CarbonAnalyticTest().test_all()

OK - test_div_by_zero
OK - test_with_known_value


# Run code:
### Using data (with python-dictionary):
```python
CarbonAnalytic(print_errors=True).calculation_from_data({
    "ISIN": "US0000000000",
    "Total Energy Use": 7000000,
    "Total CO2 Equivalents Emissions": 94972.49198,
    "Renewable Energy Purchased": 10576.00479,
    "Renewable Energy Produced": 96652.16115,
    "Carbon Credit Value": 8171.323352,
})
```
### Using list (with python-list-of-dictionaries):
```python
CarbonAnalytic(print_errors=True).calculation_from_list([
    {
        "ISIN": "US0000000000",
        "Total Energy Use": 7000000,
        "Total CO2 Equivalents Emissions": 94972.49198,
        "Renewable Energy Purchased": 10576.00479,
        "Renewable Energy Produced": 96652.16115,
        "Carbon Credit Value": 8171.323352,
        "Adjusted CO2 Total Emissions": 81902.98845798662
    },
    {
        "ISIN": "US0000000001",
        "Total Energy Use": 80000000,
        "Total CO2 Equivalents Emissions": 306900.6192,
        "Renewable Energy Purchased": 31961.79405,
        "Renewable Energy Produced": 38372.1921,
        "Carbon Credit Value": 57387.1,
        "Adjusted CO2 Total Emissions": 247545.0664717915
    },
])
```
### Using file (containing json-list-of-dictionaries):
```python
CarbonAnalytic(print_errors=True).calculation_from_file('data.json')
```

# Run tests:
```python
CarbonAnalyticTest().test_all()
```